In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
plt.rcParams['figure.max_open_warning'] = 0

In [2]:
#加载测试数据
test_dataset = pd.read_csv("test_data.csv")
print(test_dataset.shape)   #查看数据形状
test_dataset.head()  #查看数据

(160, 5)


,id,date,time,lng,lat
0,14983,2019-12-17,04:02,117.028774,21.321517
1,22597,2020-10-19,17:28,118.656037,24.509001
2,26303,2021-01-28,10:09,120.368133,24.476380
3,29235,2020-09-21,13:57,119.756953,25.670732
4,30698,2020-01-07,06:50,118.639635,23.150710


In [3]:
test_dataset.id.unique()  #160组轨迹

array([ 14983,  22597,  26303,  29235,  30698,  31064,  36050,  37063,
        59745,  66467,  71710,  74554, 102251, 110906, 111669, 119903,
       119922, 135501, 144796, 147701, 153947, 154999, 155295, 155550,
       157899, 159604, 161165, 168157, 170691, 174501, 189560, 199044,
       199252, 205315, 211423, 219106, 221614, 224481, 224855, 236861,
       263422, 269103, 275846, 278702, 284552, 287270, 293927, 296643,
       299187, 302476, 304397, 313436, 315647, 317609, 318995, 327662,
       330450, 331286, 340300, 344664, 355233, 355848, 364656, 386324,
       386656, 403467, 403917, 406509, 419396, 419817, 426634, 443857,
       454444, 462387, 465785, 470346, 473293, 479632, 481464, 498809,
       499109, 499950, 500114, 502710, 504845, 507580, 522133, 531617,
       533004, 533517, 571870, 578189, 584858, 603023, 603379, 614662,
       625648, 625947, 626023, 626884, 636329, 639323, 641549, 660165,
       660196, 678677, 679611, 680070, 687941, 691133, 691792, 693730,
      

In [4]:
from global_land_mask import globe
test_is_on_land = []
for i in test_dataset.id.unique():
    data_temp = test_dataset[test_dataset.id==i][["lng","lat"]]
    is_on_land = globe.is_land(data_temp.lat, data_temp.lng)
    test_is_on_land.append(is_on_land)
id_is_on_land=pd.DataFrame(test_is_on_land,index=test_dataset.id.unique(),columns=["is_on_land"])

In [5]:
#id_is_on_land.value_counts()

In [6]:
#数据读入
Oflow=Dataset('./current/current_hour_20191130.nc')

In [7]:
Oflow_lon = Oflow.variables['lon'][:].data.T  # 经度间隔0.005度
Oflow_lat = Oflow.variables['lat'][:].data.T   # 纬度间隔0.01度
Oflow_u = Oflow.variables['u'][:].data[0][0].T
Oflow_v = Oflow.variables['v'][:].data[0][0].T

In [8]:
Oflow_db = pd.DataFrame(np.zeros(((211, 301*4))),index=list(range(211)),columns=[["lat"]*301+["lon"]*301+["u"]*301+["v"]*301,list(range(301))*4])

In [10]:
Oflow_db["lat"] = Oflow_lat
Oflow_db["lon"] = Oflow_lon
Oflow_db["u"] = Oflow_u
Oflow_db["v"] = Oflow_v

In [ ]:
def GPS_filter(data,lon,lat,num):
    distance = 0.003*(num**0.5)
    GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
    while GPSfilter.sum().sum()!=num:
        if GPSfilter.sum().sum()>num:
            distance = distance - 0.001
        elif GPSfilter.sum().sum()<num:
            distance = distance + 0.00099
        GPSfilter = (data["lat"]>lat-distance) & (data["lat"]<lat+distance) & (data["lon"]>lon-distance) & (data["lon"]<lon+distance)
    return GPSfilter

In [ ]:
near_land = []
from tqdm import tqdm
for i in tqdm(test_dataset.id.unique()):
    data_temp = test_dataset[test_dataset.id==i][["lng","lat"]]
    GPS_sub16 = GPS_filter(Oflow_db,data_temp.head(1).lng.values[0],data_temp.head(1).lat.values[0],16)
    near_flag = True if (Oflow_db["u"][GPS_sub16].dropna(axis=0, how='all').dropna(axis=1, how='all').min().min())<-1e10 else False
    near_land.append(near_flag)
id_is_on_land["near_land"] = near_land

In [ ]:
id_is_on_land.value_counts()

In [ ]:
id_is_on_land.to_csv("id_is_on_land.csv")